<a href="https://colab.research.google.com/github/juanlu29/juanlu29/blob/contaminacionMadrid/ajusteParametrosContaminacionMadrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Optimizacion parametros para ajustar a los datos de contaminación de Madrid


In [16]:
# Importamos librerias 
import numpy as np
import random
import scipy
import scipy.linalg
from matplotlib import pyplot as plt
import seaborn as sns
from google.colab import drive
import pandas as pd
import numba
from scipy.optimize import minimize
from google.colab import drive
drive.mount('/content/drive/')


# Importamos mi libreria de procesos gaussianos
!git clone -l -s https://gist.github.com/juanlu29/ba41117769373e09d879505a76479a89 cloned-repo

# Change directory into cloned repo
%cd cloned-repo

# List repo contents
!ls

# Descargamos los datos de contaminación de Madrid

import libreriaGP as gp

# Exploramos los directorios de drive. Cambiamos directorio a donde estan los datos
%cd /content/drive/My\ Drive/datosContaminacionMadrid

# Lo pasamos a dataframes
est56NO2_df = pd.read_csv('Est56FL_CodPar08NO2_valuesonly.txt', header=0)
est24NO2_df = pd.read_csv('Est24CC_CodPar08NO2_valuesonly.txt', header=0)
est03NO2_df = pd.read_csv('Est03PlC_CodPar08NO2_valuesonly.txt', header=0)
est03O3_df  = pd.read_csv('Est03PlC_CodPar14O3_valuesonly.txt', header=0)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
fatal: destination path 'cloned-repo' already exists and is not an empty directory.
/content/drive/My Drive/datosContaminacionMadrid/cloned-repo
libreriaGP.py
/content/drive/My Drive/datosContaminacionMadrid


In [17]:
est56NO2_np = est56NO2_df.to_numpy()
est56NO2_np = est56NO2_np[0:400]

# Creamos el objeto para generar procesos gausianos
ajusteContaminacion = gp.gaussProcess(np.arange(len(est56NO2_np)),np.reshape(est56NO2_np,(len(est56NO2_np))),[0.,0.,0.,])

# Indicamos cual es el kernel y como varían en función de sus hiperparámetros
ajusteContaminacion.compCov(lambda x,y,sh,lh,sigmah : ajusteContaminacion.Kernels.exponencialCuadrada(x,y,sh,lh)+ajusteContaminacion.Kernels.ruidoBlanco(x,y,sigmah)) # Aqui especificamos los hiperparametros
funciones = np.asarray([lambda x,y,sh,lh,sigmah: ajusteContaminacion.Kernels.expCua_der_s(x,y,sh,lh),lambda x,y,sh,lh,sigmah: ajusteContaminacion.Kernels.expCua_der_l(x,y,sh,lh), lambda x,y,sh,lh,sigmah: ajusteContaminacion.Kernels.rB_der_sigma(x,y,sigmah)])
ajusteContaminacion.compDerCov(funciones)

ajusteContaminacion.Hiper = np.array([100.,365.,100])

# Entrenamos el proceso gaussiano
ajusteContaminacion.condicionarGP(*np.linspace(10.,20.,1000))


NameError: ignored